In [1]:
import gym
import gym_simpleflappy
from tqdm import tqdm
import neat
from matplotlib import pyplot as plt


env = gym.make("SimpleFlappyDistance-v0")
env.reset()

max_eps = 1000
episode = 0
steps = 0
screen_width = 288
screen_height = 512
max_speed = 10.0

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


/usr/local/lib/python3.6/dist-packages/gym/logger.py:30: UserWarning: WARN: Environment '<class 'gym_simpleflappy.envs.flappy_env.FlappyEnvDistance'>' has deprecated methods '_step' and '_reset' rather than 'step' and 'reset'. Compatibility code invoked. Set _gym_disable_underscore_compat = True to disable this behavior.
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [3]:
observation = env.reset()

In [3]:
observation, reward, done, info = env.step(1)
env.render(mode='human')
print(observation)

[247.          -9.         168.8        407.46206052 320.8
 273.20071321]


In [4]:
observation, reward, done, info = env.step(0)
env.render(mode='human')
print(observation)

[239.          -8.         164.8        407.46206052 316.8
 273.20071321]


In [2]:
def eval_genomes(individuals, config):
    """
    runs the game for each of the individuals
    :param config: config file from neat module
    :param individuals: list of individuals from one population
    :return: None
    """
    global env
    env.reset()
    nets = []
    ge = []
    
    for genome_id, genome in tqdm(individuals):
        genome.fitness = 0  # start with fitness level of 0
        net = neat.nn.FeedForwardNetwork.create(genome, config)
        nets.append(net)
        ge.append(genome)

    
    x = 0
    for net in tqdm(nets):  # Cycle for each individual
        
        observation = env.reset()  # Constructs an instance of the game
        steps = 0
        
        features = list(observation)
        features[3], features[5] = features[3]/screen_height, features[5]/screen_height
        features[2], features[4] = features[3]/screen_width, features[5]/screen_width
        features[1] /= max_speed
        output = nets[x].activate(features)  # Get input from neural network
        action = 0 if output[0] <= 0.5 else 1
        
        observation, reward, done, info = env.step(action)
        steps += 1
        
        while not done:  #  Game cycle            
            features = observation
            output = nets[x].activate(features)  # Get input from neural network
            action = 0 if output[0] <= 0.5 else 1

            observation, reward, done, info = env.step(action)  #  Make a move
            steps += 1
            if reward == -1:
                ge[x].fitness -= 10 if steps > 15 else 100
            else:
                ge[x].fitness += reward
        x += 1
           

In [3]:
def run(config_file):
    """
    runs the NEAT algorithm to train a neural network to play game.
    :param config_file: location of config file
    :return: None
    """
    config = neat.config.Config(neat.DefaultGenome, neat.DefaultReproduction,
                         neat.DefaultSpeciesSet, neat.DefaultStagnation,
                         config_file)

    # Create the population, which is the top-level object for a NEAT run.
    population = neat.Population(config)

    # Add a stdout reporter to show progress in the terminal.
    population.add_reporter(neat.StdOutReporter(False))
    stats = neat.StatisticsReporter()
    population.add_reporter(stats)
    
    checkpointer = neat.Checkpointer()
    population.add_reporter(checkpointer)

    # Run for up to 50 generations.
    winner = neat.nn.FeedForwardNetwork.create(population.run(eval_genomes, 280), config)
    observation = env.reset()  # Constructs an instance of the game    
    features = observation 
    output = winner.activate(features)  # Get input from neural network
    action = 0 if output[0] <= 0.5 else 1
    observation, reward, done, info = env.step(action)
    while not done:  #  Game cycle            
        features = observation
        output = winner.activate(features)  # Get input from neural network
        action = 0 if output[0] <= 0.5 else 1
        observation, reward, done, info = env.step(action)  #  Make a move
        env.render(mode='human')
       # show final stats
    return winner
    print('\nBest genome:\n{!s}'.format(winner))
    


In [4]:
if __name__ == '__main__':
    config_path = 'flappy-config.txt'
    winner = run(config_path)
    print(winner)

 27%|██▋       | 102/380 [00:00<00:00, 1014.54it/s]


 ****** Running generation 0 ****** 



100%|██████████| 380/380 [00:02<00:00, 156.82it/s]


Population's average fitness: 30.52368 stdev: 6.80456
Best fitness: 74.00000 - size: (21, 140) - species 173 - id 173
Average adjusted fitness: 0.148


 12%|█▏        | 91/760 [00:00<00:00, 907.15it/s]

...

Population's average fitness: 123.53125 stdev: 329.49323
Best fitness: 4536.00000 - size: (16, 51) - species 4028 - id 32169
Average adjusted fitness: 0.018


  0%|          | 0/390 [00:00<?, ?it/s]

Mean genetic distance 5.377, standard deviation 1.175
Population of 390 members in 122 species
Total extinctions: 0
Generation time: 5.722 sec (5.260 average)

 ****** Running generation 129 ****** 



100%|██████████| 390/390 [00:04<00:00, 93.40it/s] 


Population's average fitness: 172.99744 stdev: 616.36206
Best fitness: 7931.00000 - size: (14, 39) - species 4027 - id 32339

Best individual in generation 129 meets fitness threshold - complexity: (14, 39)


In [6]:
import pickle
with open('winner-ctrnn', 'wb') as f:
    pickle.dump(winner, f)